In [3]:
import numpy as np
import os
from matplotlib import pyplot as plt
from PIL import Image, ImageOps, ImageEnhance
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd
from torchvision import transforms
import torch

In [4]:
path = '.'

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [6]:
opt = {
    'image_size': 234,
    'is_grayscale': False,
}

In [7]:
def load_image(path):
    # Opening image
    im = Image.open(path).convert('L' if opt['is_grayscale'] else 'RGB')
    
    ratio = float(224) / max(im.size)
    ns = tuple([int(x * ratio) for x in im.size])
    im = im.resize(ns, Image.BICUBIC)
    ni = Image.new("L", (224, 224), "white")
    ni.paste(im, ((224 - ns[0]) // 2,
                        (224 - ns[1]) // 2))
    ni = ImageOps.expand(ni, int(np.random.randint(5, 25, size=1)), "white")
    im = ni
  
    # Enhancing image
    im = ImageEnhance.Contrast(ImageOps.autocontrast(im)).enhance(2.0)
    # Contrast adjustment
    im = ImageOps.autocontrast(im)
    im = im.resize((opt['image_size'],opt['image_size']))
    im = transforms.ToTensor()(im)
    return im

In [8]:
indices = range(len(os.listdir('train_images')))
image_list = []
for i in tqdm(indices):
    image_list.append(os.path.join('train_images',"{}.png".format(i)))

100%|█████████████████████████████████| 75233/75233 [00:00<00:00, 645733.76it/s]


In [9]:
images = torch.cat([torch.unsqueeze(load_image(image), 0) for image in tqdm(image_list)], dim=0)

100%|████████████████████████████████████| 75233/75233 [04:44<00:00, 264.50it/s]


In [10]:
torch.save(images, 'tensor.pt')

In [12]:
images.numpy().shape

(75233, 1, 234, 234)

In [ ]:
torch.save(images.clone(), 'tensor_clone.pt')